### 논문
- https://dl.acm.org/doi/pdf/10.1145/300515.300516

In [164]:
def arr_to_pi(arr):
    pi = [0]
    for num in arr:
        if num > 0:
            pi.append(2 * num - 1)
            pi.append(2 * num)
        else:
            pi.append(2 * -num)
            pi.append(2 * -num - 1)
    pi.append(len(pi))
    return pi

In [157]:
def get_bp_graph(pi):
    black_edge = [None] * len(pi)
    gray_edge = [None] * len(pi)
    breakpoint_count = 0
    
    inv = [0] * len(pi)
    for i in range(0, len(pi), 2):
        if abs(pi[i] - pi[i+1]) != 1:
            breakpoint_count += 1
            black_edge[i] = i + 1
            black_edge[i+1] = i
        inv[pi[i]] = i
        inv[pi[i+1]] = i + 1
    
    for num in range(0, len(pi), 2):
        inv0 = inv[num]
        inv1 = inv[num + 1]
        if abs(inv0 - inv1) != 1:
            gray_edge[inv0] = inv1
            gray_edge[inv1] = inv0

    return black_edge, gray_edge, breakpoint_count

In [158]:
def get_cycles(pi):
    black_edge, gray_edge, breakpoint_count = get_bp_graph(pi)
    
    visited = [False] * len(pi)
    idx_to_cycle = [None] * len(pi)
    oriented_cycle = []
    cycle_id = 0
    for start in range(0, len(pi), 2):
        if black_edge[start] is None:
            continue
        if visited[start]:
            continue
        is_oriented = False
        visited[start] = True
        cur = black_edge[start]
        on_black = False
        idx_to_cycle[start] = cycle_id
        while cur != start:
            visited[cur] = True
            idx_to_cycle[cur] = cycle_id
            next_ = black_edge[cur] if on_black else gray_edge[cur]
            if on_black and next_ < cur:
                is_oriented = True
            cur = next_
            on_black = not on_black
        cycle_id += 1
        oriented_cycle.append(is_oriented)
    cycle_count = cycle_id
        
    return black_edge, gray_edge, breakpoint_count, idx_to_cycle, oriented_cycle, cycle_count

In [170]:
def get_interleaving_graph(pi):
    black_edge, gray_edge, breakpoint_count, idx_to_cycle, oriented_cycle, cycle_count = get_cycles(pi)
    
    interleaving_graph = {i: [] for i in range(cycle_count)}
    count = [0] * cycle_count
    max_count = [0] * cycle_count
    stack1 = []
    stack2 = []
    for i in range(0, len(pi), 2):
        if idx_to_cycle[i] is None:
            continue
        max_count[idx_to_cycle[i]] += 1
    for i in range(0, len(pi), 2):
        if idx_to_cycle[i] is None:
            continue
        if count[idx_to_cycle[i]] < max_count[idx_to_cycle[i]] - 1:
            stack1.append(idx_to_cycle[i])
            count[idx_to_cycle[i]] += 1
        else:
            while count[idx_to_cycle[i]] > 0:
                last = stack1.pop()
                if last == idx_to_cycle[i]:
                    count[idx_to_cycle[i]] -= 1
                else:
                    stack2.append(last)
            while stack2:
                interleaving_cycle = stack2.pop()
                interleaving_graph[idx_to_cycle[i]].append(interleaving_cycle)
                interleaving_graph[interleaving_cycle].append(idx_to_cycle[i])
                stack1.append(interleaving_cycle)
    
    idx_to_cc = [None] * len(pi)
    cycle_to_cc = [None] * cycle_count
    cc_id = 0
    oriented_cc = []
    for i in range(len(pi)):
        if idx_to_cycle[i] is None:
            continue
        if cycle_to_cc[idx_to_cycle[i]] is not None:
            idx_to_cc[i] = cycle_to_cc[idx_to_cycle[i]]
            continue
        cycle_to_cc[idx_to_cycle[i]] = cc_id
        idx_to_cc[i] = cc_id
        is_cc_oriented = oriented_cycle[idx_to_cycle[i]]
        for interleaving_cycle in interleaving_graph[idx_to_cycle[i]]:
            cycle_to_cc[interleaving_cycle] = cc_id
            idx_to_cc[i] = cc_id
            if oriented_cycle[interleaving_cycle]:
                is_cc_oriented = True
        cc_id += 1
        oriented_cc.append(is_cc_oriented)
    cc_count = cc_id
    
    return black_edge, gray_edge, breakpoint_count, idx_to_cycle, oriented_cycle, cycle_count, interleaving_graph, idx_to_cc, cycle_to_cc, oriented_cc, cc_count

In [1]:
NOT_A_HURDLE = None
MINIMAL_HURDLE = 1
GREATEST_HURDLE = 2

def get_huddles(pi):
    black_edge, gray_edge, breakpoint_count, idx_to_cycle, oriented_cycle, cycle_count, interleaving_graph, idx_to_cc, cycle_to_cc, oriented_cc, cc_count = get_interleaving_graph(pi)
    
    first_appearance = [None] * cc_count
    last_appearance = [None] * cc_count
    appearance_count = [0] * cc_count
    reduced_cc_map = []
    for i in range(0, len(pi), 2):
        if idx_to_cc[i] is None:
            continue
        if not reduced_cc_map or reduced_cc_map[-1] != idx_to_cc[i]:
            if first_appearance[idx_to_cc[i]] is None:
                first_appearance[idx_to_cc[i]] = len(reduced_cc_map)
            last_appearance[idx_to_cc[i]] = len(reduced_cc_map)
            appearance_count[idx_to_cc[i]] += 1
            reduced_cc_map.append(idx_to_cc[i])
    
    hurdles = [NOT_A_HURDLE] * cc_count
    hurdle_count = 0
    for i in range(cc_count):
        if oriented_cc[i]:
            continue
        if appearance_count[i] == 1:
            hurdles[i] = MINIMAL_HURDLE
            hurdle_count += 1
        elif appearance_count[i] == 2 and first_appearance[i] == 0 and last_appearance[i] == len(reduced_cc_map) - 1:
            hurdles[i] = GREATEST_HURDLE
            hurdle_count += 1
        else:
            hurdles[i] = NOT_A_HURDLE

    return black_edge, gray_edge, breakpoint_count, idx_to_cycle, oriented_cycle, cycle_count, interleaving_graph, idx_to_cc, cycle_to_cc, oriented_cc, hurdles, hurdle_count

In [ ]:
# pi = [0, 5, 6, 10, 9, 15, 16, 12, 11, 7, 8, 14, 13, 17, 18, 3, 4, 1, 2, 19, 20, 22, 21, 23]
# pi = [0, 9, 10, 13, 14, 11, 12, 15, 16, 1, 2, 5, 6, 3, 4, 7, 8, 17]
# pi = [0, 3, 4, 7, 8, 5, 6, 9, 10, 13, 14, 11, 12, 15, 16, 1, 2, 17]

# arr = [3, -5, 8, -6, 4, -7, 9, 2, 1, 10, -11]
arr = [-4, -3, -5, 1, 2, 6]
pi = arr_to_pi(arr)
print(pi)

black_edge, gray_edge, breakpoint_count, idx_to_cycle, oriented_cycle, cycle_count, interleaving_graph, idx_to_cc, cycle_to_cc, oriented_cc, hurdles, hurdle_count = get_huddles(pi)
print(f'{breakpoint_count=}')
print(f'{cycle_count=}')
print(f'{hurdle_count=}')
print(breakpoint_count - cycle_count + hurdle_count)

NameError: name 'arr_to_pi' is not defined

In [154]:
def is_fortreess(pi):
    return True

In [3]:
def b(pi):
    return 0


def c(pi):
    return 0


def h(pi):
    return 0


def f(pi):
    return 1 if is_fortreess(pi) else 0


def d(pi):
    return b(pi) - c(pi) + h(pi) + (1 if is_fortreess(pi) else 0)